# 1. Import module requirement

## 1.1 Import standard Python modules

In [1]:
from collections import OrderedDict, defaultdict
import pandas as pd

## 1.2 Import openalea packages


In [2]:
from openalea.deploy.shared_data import shared_data
from openalea.mtg import *
from openalea.mtg.algo import orders
from openalea.mtg import stat
import openalea.plantgl.all as pgl

In [3]:
from openalea.core import path
from openalea.mtg import *
from openalea.deploy.shared_data import shared_data
import glob

## 1.3 Import strawberry modules

In [4]:
import openalea.strawberry

from openalea.strawberry import Rules_production
from openalea.strawberry import Visualization, visualisation2d
from openalea.strawberry import analysis, variables
from openalea.strawberry.reader import load_mtg

# 2. Import data and read MTG in files

## 2.1 Import data

La cellule ci dessous sert à changer les données. Il est donc necessaire de lui dire quelles sont les données que l'on veut importer

In [5]:
%gui qt4

files = shared_data(openalea.strawberry).glob('*.mtg')
# print files

genotypes = dict((f.namebase, f) for f in files)
print(genotypes.keys())

DIPLOIDS = ['Sicile','Nils']
OCTOPLOIDS = ['Gariguette','Darselect','Clery','Capriss','Cir107','Ciflorette']#,'friendlyfruit','friendlyfruit_varieties_finale']

diploids = dict((k,f) for k,f in genotypes.items() if k in DIPLOIDS)
# print(diploids)

octoploids = dict((k,f) for k,f in genotypes.items() if k in OCTOPLOIDS)
#print(octoploids)

[u'friendlyfruit', u'friendlyfruit_varieties', u'Ciflorette', u'Darselect', u'Gariguette', u'friendlyfruit_varieties_finale', u'Nils', u'origine_stolon_gariguetteDV_6novembre2018', u'Capriss', u'Sicile', u'Clery', u'Cir107']


## 2.2 Load one octoploid and one diploid  genotype 

A set of strawberry plant architecture has been measured and is stored in an MTG.

- We select on octoploid and/or one diploid genotype for architectural analysis.


Pour cela replacer mettre le nom de la variété voulue oct_genotype= 'nomvariete' attention les guillemets sont importante

In [6]:
oct_genotype = 'Ciflorette'

goct = MTG(octoploids[oct_genotype])
print('Octoploid  : ', str(goct))

('Octoploid  : ', 'MTG : nb_vertices=6438, nb_scales=4')


* Check number of plant present in MTG

In [7]:
# Diploid
# nb_plants = gdi.nb_vertices(scale=1)
# print(nb_plants)

# Octoploid
nb_plants = goct.nb_vertices(scale=1)
print(nb_plants)

54


## 2.3 Load whole octoploids or diploids in one big MTG

A set of strawberry plant architecture has been measured and is stored in an MTG.

- We select whole octoploids genotype for architectural analysis.

In [8]:
big_mtg = MTG()
for fn in OCTOPLOIDS:
    big_mtg = algo.union(big_mtg,MTG(octoploids[fn]))

g_octo = big_mtg
    
genotypes = set(g_octo.property("Genotype").values())
print genotypes

set(['Ciflorette', 'Darselect', 'Gariguette', 'Capriss', 'Clery', 'Cir107'])


- We select whole diploids genotype for architectural analysis.

In [9]:
# big_mtg = MTG()
# # Add missing properties
# for fn in DIPLOIDS:
#     big_mtg = algo.union(big_mtg,MTG(diploids[fn]))

# big_mtg.properties()['FLWRNUMBER']={}
# big_mtg.properties()['FLWRNUMBER_OPEN']={}

# g_diplo = big_mtg
    
# genotypes = set(g_diplo.property("Genotype").values())
# print genotypes

# 3. Strawberry Visualisation

## 3.1. Select MTG data for visualisation

- one genotype (goct or gdi)
- all genotypes (g_diplo or g_octo)

In [8]:
g=goct
g.properties()['order'] = orders(g)
g.property_names()

['LFTWD_LEFT',
 'Genotype',
 'Foliar_type',
 'LFTAR_CENTRAL',
 'LFTLG_RIGHT',
 'SAMPLING',
 'Experiment_name',
 'edge_type',
 'LFTWD_CENTRAL',
 'Architecture_date',
 'Sample_date',
 'PETLG',
 'COMMENT',
 'index',
 'label',
 'LFTAR_LEFT',
 'LFTAR_RIGHT',
 '_line',
 'Stade',
 'LFTLG_LEFT',
 'LFTWD_RIGHT',
 'FLWRNUMBER_OPEN',
 'DBI',
 'LFAR',
 'INFLOLG',
 'Plant_ID',
 'order',
 'FLWRNUMBER',
 'LFTLG_CENTRAL',
 'FLWRNUMBER_ABORTED',
 'FLWRNUMBER_CLOSED',
 'Modality']

## 3.2.  Launch 3D-visualisation

Description of argument of 3D visualisation function
- By= argument qui permet de dire qu'elle information vous voulez avoir en colonne par exemple la modalité, le génotype la date,Plant_ID
- hide_leaves, si vous voulez caché les feuille(True) ou la affichez False)

In [16]:
Visualization.plot3d(g,by=["Genotype","Sample_date"],hide_leaves=False)

Ciflorette
02/03/2015
04/12/2014
07/01/2015
13/02/2015
27/05/2015
30/03/2015


## 3.3.  Launch 2D-Visualisation

* 2D Visualization of one or a set of plants

Pour visualisation 2D:
* visualisation2d.plot2d(g, g.vertices(scale=1)[:x],dist= [y]*z) (cf ci dessous)

* arguments de la foncion:
  - [:x]  x= il sagit du nombre de plante que l'on veut représenter
  - dist= [y] x= espacement entre deux plante
  - z = nombre de plante visualisé

## TODO: Probleme avec la fonction on ne peut pas visualiser l'ensemble des plant

In [20]:
visualisation2d.plot2d(g, g.vertices(scale=1)[:54],dist= [5]*54) 

# 4. Strawberry analysis

## 4.1 Function to run the notebook on diploid or octoploid (only for one genotype)

In [14]:
# Run the notebook on Diploid or Octoploid

# Dictionnary to convert propertie names in order to run old version script
convert = dict(Stade='Stade', 
               Fleurs_ouverte='FLWRNUMBER_OPEN',
               Fleurs_avorte='FLWRNUMBER_ABORTED',
               Fleurs_total='FLWRNUMBER',
               date='Sample_date',
               Plante='Plant_ID',
               Modality="Modality"
              )

## 4.2 Extraction of data in dataframe

### 4.2.1 Extraction on plant scale

* Extraction of the most central individuals, variables (plant_variables) used;
    * Plant
    * Genotype
    * date
    * nb_total_leaves
    * nb_total_flowers
    * stolons
    * order_max
    * nb_ramification

##### TODO: Colone modalité qui permettrais de différencié les lots. 
Faire également dans la fonction calcul de l'individu central intégré une modulation par rapport à la modalité

* Donnée somme des organes totale par plant

In [15]:
# extraction of data at plant scale
df_at_plant_scale = variables.extract_at_plant_scale(g)
df_at_plant_scale.date = pd.to_datetime(df_at_plant_scale.date, dayfirst=True)
df_at_plant_scale["nb_leaf_present"]= df_at_plant_scale.nb_visible_leaves-df_at_plant_scale.nb_missing_leaves
df_at_plant_scale["total_leaf_area"]=df_at_plant_scale.nb_leaf_present*df_at_plant_scale.leaf_area
df_at_plant_scale= df_at_plant_scale[["Genotype","modality","date","plant","nb_visible_leaves","nb_total_leaves","nb_missing_leaves","nb_leaf_present","leaf_area","total_leaf_area","nb_total_flowers","nb_stolons","nb_vegetative_bud","nb_initiated_bud","nb_floral_bud","nb_inflorescence","order_max","nb_ramifications","vid"]]
df_at_plant_scale

,Genotype,modality,date,plant,nb_visible_leaves,nb_total_leaves,nb_missing_leaves,nb_leaf_present,leaf_area,total_leaf_area,nb_total_flowers,nb_stolons,nb_vegetative_bud,nb_initiated_bud,nb_floral_bud,nb_inflorescence,order_max,nb_ramifications,vid
0,Ciflorette,A,2014-12-04,1,7,9,0,7,0.00,0.00,1,3,3,1,1,1,1,0,1
1,Ciflorette,A,2014-12-04,2,8,13,0,8,116.11,928.88,0,2,2,2,5,0,1,1,43
2,Ciflorette,A,2014-12-04,3,5,8,0,5,88.12,440.60,0,0,2,2,3,1,1,0,120
3,Ciflorette,A,2014-12-04,4,4,8,0,4,88.33,353.32,0,0,1,2,5,0,0,0,172
4,Ciflorette,A,2014-12-04,5,6,9,0,6,91.12,546.72,0,2,0,2,5,0,0,0,235
5,Ciflorette,A,2014-12-04,6,8,12,0,8,64.25,514.00,0,2,4,1,4,0,0,0,309
6,Ciflorette,A,2014-12-04,7,8,11,0,8,88.38,707.04,0,1,2,1,6,0,0,0,374
7,Ciflorette,A,2014-12-04,8,5,7,0,5,87.95,439.75,0,0,2,1,4,0,0,0,454
8,Ciflorette,A,2014-12-04,9,10,16,0,10,108.07,1080.70,0,2,2,3,6,0,1,1,508
9,Ciflorette,A,2015-01-07,1,10,10,0,10,58.73,587.30,10,2,0,1,6,1,0,0,590


In [24]:
# Exctract data in csv (Excel)
# indiquer le chemin ou l'on souhaite l'enregitrer entre guillement et ajouter le nom fichier.csv
#exemple path_or_buf='chemin d'enregistrement/nom_fichier.csv' ()

df_at_plant_scale.to_csv(path_or_buf='c:/Users/mlabadie/Documents/GitHub/strawberry/Rscript/extraction_plant_scale.csv', sep=';')

* Identification du plant median

In [16]:
# Data of the central plants
df_individuals = analysis.median_individuals(df_at_plant_scale)
df_individuals = df_individuals.sort_values(by=['Genotype',"modality",'date'])

df_individuals

,Genotype,modality,date,plant,nb_visible_leaves,nb_total_leaves,nb_missing_leaves,nb_leaf_present,leaf_area,total_leaf_area,nb_total_flowers,nb_stolons,nb_vegetative_bud,nb_initiated_bud,nb_floral_bud,nb_inflorescence,order_max,nb_ramifications,vid
4,Ciflorette,A,2014-12-04,5,6,9,0,6,91.12,546.72,0,2,0,2,5,0,0,0,235
15,Ciflorette,A,2015-01-07,7,10,10,0,10,57.68,576.80,11,3,0,0,7,1,0,0,1146
25,Ciflorette,A,2015-02-13,8,17,19,0,17,76.47,1299.99,32,1,4,2,9,4,1,2,2300
34,Ciflorette,A,2015-03-02,8,19,21,0,19,108.50,2061.50,34,2,5,0,8,4,2,2,3510
42,Ciflorette,A,2015-03-30,7,26,27,0,26,81.68,2123.68,39,1,7,0,8,5,2,4,4588
53,Ciflorette,A,2015-05-27,9,35,39,0,35,128.87,4510.45,62,7,6,0,10,10,4,2,6279


In [16]:
# Exctract data in csv (Excel)
# indiquer le chemin ou l'on souhaite l'enregitrer entre guillement et ajouter le nom fichier.csv
#exemple path_or_buf='chemin d'enregistrement/nom_fichier.csv' ()
df_individuals.to_csv(path_or_buf='c:/Users/mlabadie/Documents/GitHub/strawberry/Rscript/centralindividuals.csv', sep=';')

* 2D visualisation of the most central individuals

In [17]:
# selection of vid of median individuals
pids = list(df_individuals.vid)
n = len(pids)

# Plot 2D- visualisation of the most central plant
# dist= [x]*n, x= distance entre 2 plants
visualisation2d.plot2d(g, pids, dist=[6]*n)


# TODO:Pensez a permettre une visualisation 3D de l'individu central

### 4.2.2 Data extraction on module scale

**Variables used (module_variables):**
- nb_visible_leaves # Nombre de feuille developpe
- nb_foliar_primordia #Nombre de primordia foliaire
- nb_total_leaves #Nombre total de feuille
- nb_open_flowers #Nombre de fleurs ouverte
- nb_aborted_flowers #Nombre de fleurs avorté
- nb_total_flowers #Nombre total de Fleurs
- nb_vegetative_buds
- nb_initiated_buds
- nb_floral_buds
- nb_stolons
- type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
- Crown_status (1: vegetatif, 2: initiated, 3: Floral bud, 4: Inflorescence)

In [ ]:
# extraction of data at module scale
df_at_module_scale = variables.extract_at_module_scale(g)
df_at_module_scale = df_at_module_scale.sort_values(by=["Genotype","date","modality",'plant',"order"])
df_at_module_scalemean= df_at_module_scale.groupby(["Genotype","modality","order"]).mean().round()
df_at_module_scale


In [ ]:
# Exctract data in csv
df_at_module_scale.to_csv(path_or_buf='c:/Users/mlabadie/Documents/GitHub/strawberry/Rscript/2-Friendlyfruit_Project/friendlyfruit_varieties_module_scale.csv', sep=';')

## 4.3 Data sequence extractions

### 4.3.1 Module scale

* **Index:**
    - real Order (Index) for recouvrant extraction
    - Apparent Order for non recouvrant extraction

* **Variables:**
    - nb_visible_leaves : count F label in components
    - nb_foliar_primordia: count f si le module commence par F
    - nb_total_leaves: sum(F+f) du module
    - Stolon Numbers: count s label in components and attach at the order -1 
    - nb_open_flowers: g.property('Fleurs_ouverte') at scale 3
    - nb_aborted_flowers: g.property('Fleurs_avorte') at scale 3
    - nb_total_flowers : g.property('Fleurs_total') at scale 3
    - No_vegetative_bud: count bt if g.property(Stade) == none or 17 or 18 or 19
    - No_iniated_bud: count bt if g.property(Stade) == A
    - No_Floral_bud: count ht
    - No_Inflorescence: count HT
 
 
* **Variable qualitative:**
    - type of crown
        - label == T (1) : principal crown
        - parent(component_roots()[0]) : if successor() == F : branch_crown (3)
        - elif in (HT, ht, bt) extension crown (2)
    - inflorescence_type:
        - vegetative (1): label==bt g.property(Stade)== none or 17 or 18 or 19
        - initiated (2): label== bt if g.property(Stade) == A
        - floral (3): label== ht or HT


* **General Variables:**
    - Genotype: g.property('Genotype') at scale 1
    - Date: g.property('date') at scale 1
    - Plante Number: g.property('plante') at scale 1

In [ ]:
# Run the notebook on Diploid or Octoploid
IS_DIPLOID = False

if IS_DIPLOID:
    g = g_diplo
else:
    g = g_octo



print(g.property_names())
convert = dict(Stade='Stade', 
               Fleurs_ouverte='FLWRNUMBER_OPEN',
               Fleurs_avorte='FLWRNUMBER_ABORTED',
               Fleurs_total='FLWRNUMBER',
               date='Sample_date',
               Plante='Plant_ID',
              )

# if not IS_DIPLOID:
#     g.properties()['Sample_date'] = g.property('Architecture_date')

def dates_diploid():
    l = '11-05-2016', '17-05-2016', '01-06-2016', '20-06-2016'
    return dict(zip(l, range(1, len(l)+1)))

def dates_octoploid():
    d = {'10-12-2014':1,'07-01-2015':2,'15-02-2015':3,'02-03-2015':4,'03-04-2015':5,'27-05-2015':6,'04-12-2014':1,'13-02-2015':3,
         '30-03-2015':5,'08-01-2015':2,'11-02-2015':3,'04-03-2015':4,'02-04-2015':5,'20-05-2015':6,'09-01-2015':2,'06-03-2015':4,
         '12-02-2015':3,'19-05-2015':6}
    return d

def dates():
    if IS_DIPLOID:
        return dates_diploid()
    else: 
        return dates_octoploid()


def property(g, name):
    """ We can change the name of the MTG properties without changing the code"""
    return g.property(convert.get(name, name))
    

In [ ]:
# apparent_module: all module which contains a visible leaf F
def visible_modules(g=g):
    return [v for v in g.vertices_iter(scale=2) if g.label(g.component_roots_iter(v).next()) == 'F']

In [ ]:
# add property in MTG, apparant module (True) or not
modules = visible_modules()
_visible = {}
for m in modules:
    _visible[m] = True
g.properties()['visible'] = _visible

In [ ]:
"""Index

TODO: Replace dict by functions with specific traversal.

"""
#order scale 2: order at module scale
orders = algo.orders(g, scale=2)

"""Variable quantitative"""
#function which count all F
def nb_visible_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='F')

#function which count all f
def nb_foliar_primordia(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='f')

#function which count all f+F
def nb_total_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid) in ('f', 'F'))

""" nb_stolon"""
#function count stolon
def nb_stolons(v, g=g):
    def nb_stolon(vid, g=g):
        return sum(1 for cid in g.components(vid) if g.label(cid)=='s') 
    return sum(nb_stolon(ch) for ch in g.children(v))

#function return number of open flower
def nb_open_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_ouverte')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of aborted flower
def nb_aborted_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_aborted')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of total flower
def nb_total_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_total')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))


"""Return the No vegetative bud
    
Algorithm:
if labet is bt then stage is 17,18,19 or None
count number of bt and attach at the parent order
    """
# function return number of vegetative buds
def nb_vegetative_buds(vid, g=g):
    stages= property(g, 'Stade')

    def nb_vegetative(v):
        cid = g.component_roots(v)[0]
        
        return sum(1 for cid in g.components(v) if g.label(cid)=='bt' and stages.get(cid) in (None,'17','18','19'))
    
    return sum(nb_vegetative(ch) for ch in g.children(vid))

 
""" Return the No initiated bud"""

def nb_initiated_buds(vid, g=g):
    
    stages= property(g, 'Stade')
    
    def nb_init(v):
        return sum(1 for cid in g.components(v) if (g.label(cid)=='bt') and (stages.get(cid)=='A'))

    return sum(nb_init(ch) for ch in g.children(vid))


""" Return the No Floral bud"""
def nb_floral_buds (vid, g=g):
    visibles = property(g, 'visible')
    def nb_floral(v):
        return sum(1 for cid in g.components(v) if g.label(cid)=="ht" )
    return sum(nb_floral(ch) for ch in g.children(vid) if ch not in visibles)


""" Qualitative variables"""
def type_of_crown(vid, g=g):
    """ Returns the type of crown.
    
    Definition of type of crown (1, 2, 3):
     - principal crown (1): label == T 
     - branch_crown (3)
         parent(component_roots()[0]) : if successor() == F  
     - extension_crown (2): contains(HT, ht, bt) 
     - error (4)
     
    """
    if g.label(vid) == 'T':
        return 1
    else:
        cid = g.component_roots_iter(vid).next()
        pid = g.parent(cid)
        sid = g.Successor(pid)
        #print sid
        if g.label(sid) in ('F', 'f'):
            return 3
        elif g.label(sid) in ('bt', 'ht', 'HT'):
            return 2
        else:
            # ERROR !!!
            print g[cid], g[g.complex_at_scale(cid, scale=1)]
            return 4
            
def Crown_status(vid, g=g):
    """ Returns the type of inflorescence
    
    :Algorithms:
    if label is bt then
        - if stage is 17, 18, 19 or None, => vegetative (1)
        - if stage is A => initiated (2)
        - if stage is other => non defined (pourri, avorté, coupé) (-1)
     - Terminal vegetative bud (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - Terminal initiated bud (2): label== bt if g.property(Stade) == A
     - Terminal Floral bud (3): label==ht
     - Inflorescence Terminal (4): label== HT
     - runner (5): label = s
     
    """
    stages = property(g,'Stade')
    # select s, ht, HT et bt
    for cid in g.components(vid):
        if g.label(cid) in ('s', 'ht', 'HT', 'bt'):
            label = g.label(cid)
            if label == 'ht':
                return 3
            elif label == 'HT':
                return 4
            elif label == 'bt':
                stage = stages.get(cid)
                if stage == 'A':
                    return 2
                elif stage in (None, '17', '18', '19'):
                    return 1
            elif label == 's':
                return 5
            return -1

def nb_inflorescence (Vid, g):
    return sum(1 for cid in g.components(Vid) if g.label(cid)=='HT')
    
def genotype(vid, g=g):
    d = {'Capriss':4, 'Ciflorette':2, 'Cir107':6, 'Clery':3, 'Darselect':5, 'Gariguette':1,
         'Nils': 1, }
    
    cpx = g.complex_at_scale(vid, scale=1)
    _genotype = property(g, 'Genotype')[cpx]
    return d[_genotype]

def plant(vid, g=g):
    
    cpx = g.complex_at_scale(vid, scale=1)
    return property(g, 'Plante')[cpx]

def date(vid, g=g):
    d = dates()
    cpx = g.complex_at_scale(vid, scale=1)
    _date = property(g, 'date')[cpx]
    return(d[_date])

In [ ]:
# Extract sequences

#seqs = stat.extract_extremities(g, scale=2)
#seqs = stat.extract_axes(g, scale=2)
def is_extremities(g, v):
    visibles = property(g, 'visible')
    ext = [c for c in g.children(v) if c in visibles and not is_axis_root(g, c)]
    return True if not ext else False

def extract_extremities(g, modules):
    """ Extract sequences that connect from leaves modules to the root ones of each plant. """
    leaves = [m for m in modules if is_extremities(g, m)]
    seqs = [list(reversed([vid for vid in algo.ancestors(g,lid)])) for lid in leaves]
    return seqs

def is_axis_root(g, vid):
    cid = g.component_roots_iter(vid).next()
    pid = g.parent(cid)
    sid = g.Successor(pid)
    if g.label(sid) not in ('bt', 'ht', 'HT'):
        return True
    else:
        return False
    

def apparent_axis(g, vid):
    visibles = property(g, 'visible')
    v = vid
    while v is not None:
        yield v
        vtx = v; v = None
        for vid in g.children(vtx):
            if (vid in visibles) and (not is_axis_root(g, vid)):
                v = vid
            
def extract_apparent_axes(g, modules):
    # Extract all the vertices with edge_type == '+'
    vids = [m for m in modules if is_axis_root(g, m)]
    
    seqs = [ list(apparent_axis(g, vid)) for vid in vids]
    return seqs


In [ ]:
seqs1 = extract_apparent_axes(g, modules)
seqs2= extract_extremities(g,modules)
seqs = seqs1
len (seqs)

In [ ]:
# Index: order
VertexIdentifiers = [['-'.join(str(orders[j]) for j in seq[0:i+1]) for i, vid in enumerate(seq)] for seq in seqs]

# Variables
variable = OrderedDict()
variable['nb_visible_leaves'] = nb_visible_leaves # Nombre de feuille developpe
variable['nb_foliar_primordia'] = nb_foliar_primordia #Nombre de primordia foliaire
variable['nb_total_leaves'] = nb_total_leaves #Nombre total de feuille
variable['nb_open_flowers'] = nb_open_flowers #Nombre de fleurs ouverte
variable['nb_aborted_flowers'] = nb_aborted_flowers #Nombre de fleurs avorté
variable['nb_total_flowers'] = nb_total_flowers #Nombre total de Fleurs
variable['vegetative_bud'] = nb_vegetative_buds
variable['Initiated_bud']= nb_initiated_buds
variable['floral_bud']= nb_floral_buds
variable['stolons']= nb_stolons
variable['type_of_crown'] = type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
variable['Crown_status'] = Crown_status # Status du coeurs ("vegeatatif:1, Iniated: 2, Floral: 3)


variable['genotype'] = genotype # TODO
variable['date'] = date # TODO
variable['plant'] = plant

In [ ]:
variable_names = variable.keys()
print variable_names

final_seqs = [[[f(vid) for f in variable.values()] for vid in seq] for seq in seqs]

In [ ]:
final_seqs

In [ ]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, VertexIdentifiers)

In [ ]:
print sequence_txt

In [ ]:
f = open('c:/Users/mlabadie/Documents/GitHub/strawberry/Rscript/strawberry_archi_module_nonoverlaping_path.seq', 'w') #seq1
f.write(sequence_txt)
f.close()

### 4.3.2 At Phytomer scale

#### 1. Index
Index is node on apparent trunk axis  
VARIABLE 1: INT  # module order starting at 0  

#### 2. Qualitative variable
VARIABLE 2: INT  

- types of axillary production 
    - -1: missing bud; 
    - 1: stolon; 
    - 2: vegetative bud (stages 17, 18, 19); 
    - 3: initiated bud (stage A); 
    - 4: rotten, dry or aborted bud; 
    - 5: floral bud; 
    - 6: branch crown; 
    - 7: inflorescence  
 
On each nodes identify the type of ramification:
ramif-type: 

- if is invisible modules==F take form of:   
    - stolon (s): 1, 
    - vegetative bud(bt, at stage None, 17,18,19):2,
    - initiated bud, (bt, at stage A):3, 
    - floral bud(ht):4 
     
- if is visible module==T:  
    - branch crown (complex ramification):5

#### 3. Global variables

- VARIABLE 3: INT  # genotype (1: Gariguette; 2: Ciflorette; 3: Clery; 4: Capriss; 5:Darselect; 6: Cir107)
- VARIABLE 4: INT  # date (1: mid-December; 2: early January; 3: mid-February; 4: early March; 5: early April; 6: end May/early June)
- VARIABLE 5: INT  # plant index


#### 4. Quantitative variable

if branch crown(5):
- on principal path:
    - No. modules
    - No. developed leaves (F)
    - No. primordia leaves (f)
    - No. total leaves (f+F)
    - No. open flower, g.property("open_flower")
    - No. aborded flower, g.property("aborded_flower")
    - No. total flower, g.property("total flower")
    
- on whole branch crown:
    - No. branch crown carried sum(No. visible modules if modules is_axis_root==T )  
    - No. extension crown carried sum(No. visible module if modules is_axis_root==F) 
    - No. total modules carried 
        - sum(No.visible modules) 
    - No. developed leaves carried 
        - sum(for vid in g if g.label=="F" and if visible module==T) 
    - No. primordia leaves carried sum(f, if visible module==T)
    - No. total leaves carried sum(F+f, if visible module==T)
    - No. vegetative buds carried sum(bt at stage None, 17,18,19 if visible modules ==F) 
    - No. initiated buds carried sum(bt at stage A if visible modules == F) 
    - No. floral buds carried sum(ht if visible modules==F)
    - No. Inflorescence carried sum(HT if visible module==T)
    - No. open flowers carried sum(g.property("open_flowers"), if visible modules==T)
    - No. aborded flowers carried sum(g.property("aborded_flower"if visible module==T)
    - No. total flowers carried sum(g.property("total_total flowers" if visible module==T)
    - No. stolon carried sum(s, if visible module ==F)
 

In [ ]:
from itertools import chain
from openalea.mtg import algo, traversal

In [ ]:
# Extract tall the modules of each plant at scale 2
#roots = modules
# assert set(g.label(r) for r in roots) == set(['T']) 

trunks = [ list(chain(*[(v for v in algo.axis(g,m, scale=3) if g.label(v) in ('F', 'f')) for m in axis])) for axis in extract_apparent_axes(g, modules)]
# print trunk

# define the apparent order on the MTG
vertex_ids = [[str(rank) for rank, v in enumerate(trunk)] for trunk in trunks]

ranks = dict()
roots3 = g.component_roots_at_scale(g.root, scale=3)
for r in roots3:
    for v in traversal.pre_order2(g, r):
        ranks[v] = 0 if g.parent(v) is None else ranks[g.parent(v)]+1
        
vertex_ids = [[ranks[v] for v in trunk] for trunk in trunks]

# extract a sequence of F for each module
#print vertex_ids

In [ ]:
orders3 = algo.orders(g, scale=3)

def branching(g, v):
    sid = g.Successor(v)
    if g.label(sid) in ('bt', 'ht', 'HT'):
        return None
    
    kids = [c for c in g.children(v) if g.edge_type(c) =='+' and g.label(c) == 'F' ]
    if not kids:
        return None
    
    return kids[0]

def branching_type(vid, g=g):
    """ Returns the type of branching
    
    :Algorithms:
    
    if module is visible:
        - branch crown (complex ramification):6
        - inflorescence : 7

    if module is invisible:
        - stolon (s): 1,
        - vegetative bud(bt, at stage None, 17,18,19):2,
        - initiated bud, (bt, at stage A):3,
        - aborted or roten or dried bud: 4
        - floral bud(ht):5

     
    """

    cpx = g.complex(vid)
    nxid = g.node(cpx) 
    if nxid.visible:
        if g.label(vid) == 'HT':
            return 7
        else:
            return 6
    
    # select s, ht, et bt
    for cid in nxid.components():
        label = cid.label
        if label in ('s', 'ht', 'bt'):
            if label == 's':
                return 1
            elif label == 'bt':
                stage = cid.Stade
                if stage in (None, '17', '18', '19'):
                    return 2
                elif stage == 'A':
                    return 3
                elif stage in ('pourri', 'aborted', 'dried'):
                    return 4
            elif label  == 'ht':
                return 5
    else:
        print 'ERROR: ', cpx, nxid.complex().Genotype, nxid.properties()

#[[branching(g, v) for v in trunk] for trunk in trunks]

# Extract on a vertex the tree of modules
def module_tree(v, g):
    visibles = g.property('visible')
    cpx = g.complex(v)
    return [m for m in traversal.pre_order2(g, cpx) if m in visibles]

In [ ]:
def module_axis(v, g=g):
    cpx = g.complex(v)
    return list(apparent_axis(g, cpx))

def nb_modules_axis(v, g=g):
    return len(module_axis(v, g=g))

def nb_visible_leaves_axis(v, g=g):
    return sum(nb_visible_leaves(m, g) for m in module_axis(v, g=g))

def nb_foliar_primordia_axis(v, g=g):
    return sum(nb_foliar_primordia(m, g) for m in module_axis(v, g=g))

def nb_total_leaves_axis(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_axis(v, g=g))

def nb_open_flowers_axis(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_axis(v, g=g))

def nb_aborted_flowers_axis(v, g=g):
    return sum(nb_aborted_flowers(m, g) for m in module_axis(v, g=g))

def nb_total_flowers_axis(v, g=g):
    return sum(nb_total_flowers(m, g) for m in module_axis(v, g=g))

def nb_vegetative_buds_axis(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_axis(v, g=g))

def nb_initiated_buds_axis(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_axis(v, g=g))

def nb_floral_buds_axis(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_axis(v, g=g))

def nb_stolons_axis(v, g=g):
    return sum(nb_stolons(m, g) for m in module_axis(v, g=g))


####################################################
def nb_total_module_tree(v, g=g):
    return len(module_tree(v, g))

def nb_branching_tree(v, g=g):
    return sum(1 for m in module_tree(v, g) if is_axis_root(g, m))

def nb_extension_tree(v,g=g):
    return sum(1 for m in module_tree(v,g) if not is_axis_root(g, m))

def nb_visible_leaves_tree(v, g=g):
    return sum(nb_visible_leaves(m,g) for m in module_tree(v, g))

def nb_primordia_leaves_trees(v, g=g):
    return sum(nb_foliar_primordia(m,g) for m in module_tree(v, g))

def nb_total_leaves_trees(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_tree(v, g))

def nb_vegetative_buds_tree(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_tree(v, g))

def nb_initiated_buds_tree(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_tree(v, g))

def nb_floral_buds_tree(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescences(m, g) for m in module_tree(v, g))

def nb_open_flowers_tree(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_tree(v, g))

def nb_aborted_flowers_tree(v , g=g):
    return sum(nb_aborted_flowers(m, g=g) for m in module_tree(v, g))

def nb_total_flowers_tree(v, g=g):
    return sum(nb_total_flowers(m, g=g) for m in module_tree(v, g))

def nb_stolon_tree(v, g=g):
    return sum(nb_stolons(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescence (m, g) for m in module_tree(v, g))

##################################################
def genotype_axis(v, g=g):
    return genotype(g.complex(v), g)
def date_axis(v, g=g):
    return date(g.complex(v), g) #Correction my_date en date
def plant_axis(v, g=g):
    return plant(g.complex(v), g)
# test
#trunk = trunks[1]
#[nb_stolon_tree(branching(g, v)) for v in trunk if branching(g, v)]

In [ ]:
# Variables
variables = OrderedDict()

# on principal ramification path
#variables["branching_type"]= branching_type
variables['nb_module_axis'] = nb_modules_axis
variables['nb_visible_leaves_axis'] = nb_visible_leaves_axis # Nombre de feuille developpe
variables['nb_foliar_primordia_axis'] = nb_foliar_primordia_axis #Nombre de primordia foliaire
variables['nb_total_leaves_axis'] = nb_total_leaves_axis #Nombre total de feuille
variables['nb_vegetative_buds_axis']= nb_vegetative_buds_axis # nombre de vegetative bud 
variables['nb_initiated_buds_axis']= nb_initiated_buds_axis # nombre of initiated buds
variables['nb_floral_buds_axis']= nb_floral_buds_axis #nb of floral buds
variables['nb_open_flowers_axis'] = nb_open_flowers_axis #Nombre de fleurs ouverte
variables['nb_aborted_flowers_axis'] = nb_aborted_flowers_axis #Nombre de fleurs avorté
variables['nb_total_flowers_axis'] = nb_total_flowers_axis #Nombre total de Fleurs
variables['nb_stolons_axis'] = nb_stolons_axis

#on whole ramification 
variables["nb_branch_crown_tree"]= nb_branching_tree # sum(No. visible modules if modules is_axis_root )
variables["nb_extenstion_crown_tree"]= nb_extension_tree # sum(No. visible module if modules nit is_axis_root)
variables["nb_total module_tree"]= nb_total_module_tree # sum(visible modules)
variables["nb_visible_leaves_tree"]= nb_visible_leaves_tree # sum(F, if visible module==T)
variables["nb_primordia_leaves_trees"]=nb_primordia_leaves_trees #sum(f, if visible module==T)
variables["nb_total_leaves_trees"]=nb_total_leaves_trees #sum(F & f , if visible module==T)
variables["nb_vegetative_buds_tree"]= nb_vegetative_buds_tree# sum(bt if stages in (None,'17','18','19') and visible module==F)
variables["nb_initiated_buds_tree"]= nb_initiated_buds_tree# sum(bt if stages=='A' and visible module==F)
variables["nb_floral_buds_tree"]= nb_floral_buds_tree# sum(ht if visible module==F)
variables["nb_inflorescence_tree"]= nb_inflorescences_tree# sum(HT if visible module==T)
variables["nb_open_flowers_tree"]= nb_open_flowers_tree# sum(g.property('Fleurs_ouverte') if visible module==T)
variables["nb_aborted_flowers_tree"]= nb_aborted_flowers_tree# sum(g.property('Fleurs_aborted') if visible module==T)
variables["nb_total_flowers_tree"]= nb_total_flowers_tree# sum(g.property('Fleurs_total') if visible module==T)
variables["nb_stolon_tree"]= nb_stolon_tree# sum(s if visible modules ==F)

# add supplemental variables
#variables No. Inflorescence
#variables No. Inflorescence crown bearing by ramification
#variables No Vegetative crown bearing by ramification
#variables sum initiated crown bearing by ramification
#variable sum floral crown bearing by ramification

#Global variable
global_variables = OrderedDict()
global_variables['genotype'] = genotype_axis # TODO
global_variables['date'] = date_axis # TODO
global_variables['plant'] = plant_axis

# Extract variables fo Inflorescences
inflo_variables = OrderedDict()
inflo_variables['nb_total_flowers_axis'] = nb_total_flowers 
inflo_variables['nb_open_flowers_axis'] = nb_open_flowers #Nombre de fleurs ouverte
inflo_variables['nb_aborted_flowers_axis'] = nb_aborted_flowers #Nombre de fleurs avorté


In [ ]:
final_seqs = []
# add if not ramif:
def is_inflo(g, vid):
    return True if g.label(vid) =='HT' else False

def filter_hamp_branch_crown(g, vid):
    """Extract first ramification node or inflo. """
    inflo = [c for c in g.children(vid) if is_inflo(g, c)]
    if inflo:
        return inflo[0]
    
    ramif = [c for c in g.children(vid) if (g.edge_type(c) =='+')]
    if not ramif:
        return None
    if len(ramif) > 1: 
        print vid, genotype(vid), 'line ', g.node(vid)._line, len(ramif)
    return ramif[0]


for trunk in trunks:
    seq = []
    for vid in trunk:
        vars = [orders3[vid]]
        # Extract the next vertex that is the root of a sub-tree
        bid = filter_hamp_branch_crown(g, vid)
        if bid is None:
            vars.append(-1)
        else:
            vars.append(branching_type(bid, g))

        inflo = is_inflo(g, bid) 
        branch_id = branching(g, vid)
        vars.extend([f(vid) for f in global_variables.values()])
        
        """
        for name in variables:
            if inflo and name in inflo_variables:

                f = inflo_variables[name]
                vars.append(f(g.complex(bid)))
                print 'INFLO ', f(g.complex(bid))
            elif branch_id is not None: 
                f = variables[name]
                vars.append(f(branch_id))
            else:
                vars.append(0)
        """
            #vars.extend([f(branching(g, vid)) if branching(g, vid) is not None elif inflo_variables[name] if is_inflo(g, vid) else 0 for f in variables.values()])
        seq.append(vars)
    final_seqs.append(seq)

#[[f(branching(g, vid)) if branching(g, vid) is not None else -1 for f in variables.values()] for vid in trunk] for trunk in trunks]
final_seqs                          

In [ ]:
variables
variable_names = ['order', 'branching_type']+global_variables.keys() #+inflo_variables.keys()+variables.keys()
variable_names

In [ ]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, vertex_ids)
print sequence_txt

In [ ]:
f = open('strawberry_archi_node_trunk_path.seq', 'w') #seq
f.write(sequence_txt)
f.close()

In [ ]:
l=range(10)

In [ ]:
l.sort()


In [ ]:
g.property('LFAR')

In [ ]:
g.property('LFTLG_LEFT')